In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# print("일사량 => ", SOLAR)
SOLAR ='reset_match'

### 모델링

##### LinearRegression, SVM(SVC, SVR), Gradientboosting, Decisiontree, Randomforest

### 모듈 불러오기

#### 규칙 
    훈련, 정확도
    1) 기업데이터의 일사량으로 만 훈련, 스코어
    2) 기업데이터의 일사량, 기상청의 환경데이터 포함 훈련, 스코어
    -> 더 높은 스코어를 가진 규칙 채택

    예측,
    1-1) 훈련모델 예측 -> 기상청의 일사량으로 예측
    1-2) 훈련모델 예측 -> 기상청의 일사량을 보정 후 예측

    2-1) 훈련모델 예측 -> 기상청의 일사량, 기상청의 환경데이터로 예측
    2-2) 훈련모델 예측 -> 기상청의 일사량을 보정 후, 기상청의 환경데이터로 예측


        ### 결과 :: 
            1) 이 높다 -> 환경데이터를 사용하는 것이 무의미 
            2) 가 높다 -> 환경데이터를 사용하는 것 의미 있다.

            최상의 결과 : 2-2가 가장 정확한 것


In [3]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import os 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go # 그래프 객체 생성
from plotly.subplots import make_subplots

os.getcwd()

'C:\\Users\\wjdal\\OneDrive\\바탕 화면\\PROJECT_ENITT\\Prediction_Solar_Generation\\NoteBook'

In [4]:
# banpo = pd.read_csv('../Data/Final_Data/banpo1_power_weather_df.csv', encoding="utf8")

In [5]:
# banpo

In [6]:
def mean_absolute_percentage_error(a, b):
    
    a_set, b_set = np.array(a), np.array(b)
    
    a_list= []
    b_list = []
    for i, j in zip(a_set, b_set):
        for z in i:
            a_list.append(z)
        b_list.append(j)
    # print(a_list)
    df1 = pd.Series(a_list)
    df2 = pd.Series(b_list)

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=df1.index, y=df1,  mode='lines', name="true", line=dict(color='blue')), secondary_y = False)
    fig.add_trace(go.Scatter(x=df2.index, y=df2, mode='lines', name="pred", line=dict(color='red')), secondary_y = False)
    fig.update_layout(width=1000, height=500,)
    fig.show()

    return np.mean(np.abs((a_set - b_set) / a_set)) * 100

### load_dataset

In [7]:
df = pd.read_csv("../Data/Final_Data/total_enitt_data2.csv", encoding="utf-8")
# df = pd.read_csv("./마지막데이터/banpo1_power_weather_df.csv", encoding="utf-8")
df = df[~(df["합계 일사량(MJ/m2)"].isna() == True)]


df_need_1 = df[["env_slopesolar", "watt", "reset_match", "match_solar"]]
df_need_2 = df[["env_slopesolar", "최저기온(°C)", "최고기온(°C)", "일강수량(mm)", "평균 풍속(m/s)","평균 상대습도(%)", "가조시간(hr)","평균 전운량(1/10)",  "watt", "reset_match","match_solar"]]
df_need_2.columns = ["env_slopesolar", "최저기온", "최고기온", "일강수량", "평균풍속", "평균상대습도", "가조시간", "평균전운량", "watt", "reset_match", "match_solar"]

df_need_1
df_need_2 = df_need_2.fillna(0)



### train_test_split

In [8]:
from sklearn.model_selection import train_test_split

def split_set(df):
    x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1:], test_size=0.2, shuffle=True , random_state = 42)

    return x_train, x_test, y_train, y_test
 

    1) LinearRegression

In [9]:


# import statsmodels.formula.api as smf

# print("----------ols----------")

# lr_model = smf.ols(formula = "watt~env_slopesolar", data=df_need_1)
# lr_model = lr_model.fit()
# print(lr_model.summary())

# lr_model2 = smf.ols(formula = "watt~env_slopesolar + 최저기온 + 최고기온 + 일강수량 + 평균풍속 + 평균상대습도 + 가조시간 + 평균전운량", data=df_need_2)
# lr_model2 = lr_model2.fit()
# print(lr_model2.summary())




In [10]:

from sklearn.linear_model import LinearRegression

def model(solar, name):
    print("----------LinearRegression----------")


    rd_df_x = df_need_2[[SOLAR]]
    rd_df_y = df_need_2[["watt"]]

    rd_df_x_train, rd_df_x_test, rd_df_y_train, rd_df_y_test = train_test_split(rd_df_x, rd_df_y, test_size=0.2, shuffle=True, random_state = 42)

    model = LinearRegression()
    model.fit(rd_df_x_train, rd_df_y_train)
    train_score1 = model.score(rd_df_x_train, rd_df_y_train)
    test_score1 = model.score(rd_df_x_test, rd_df_y_test)
    pred1 = model.predict(rd_df_x_test)
    # acc_score1 = accuracy_score(rd_df_y_test, pred1)

    print("훈련정확도 : ", train_score1)
    print("예측정확도 : ", test_score1)
    # print("acc_score : ", acc_score1)


    rd_df_x = df_need_2[[SOLAR, "최저기온", "최고기온", "일강수량", "평균풍속", "평균상대습도", "가조시간", "평균전운량"]]
    rd_df_y = df_need_2[["watt"]]

    rd_df_x_train, rd_df_x_test, rd_df_y_train, rd_df_y_test = train_test_split(rd_df_x, rd_df_y, test_size=0.2, shuffle=True, random_state = 42)

    model = LinearRegression()
    model.fit(rd_df_x_train, rd_df_y_train)
    train_score2 = model.score(rd_df_x_train, rd_df_y_train)
    test_score2 = model.score(rd_df_x_test, rd_df_y_test)
    pred2 = model.predict(rd_df_x_test)
    # acc_score2 = accuracy_score(rd_df_y_test, pred2)

    print("훈련정확도2 : ", train_score2)
    print("예측정확도2 : ", test_score2)
    # print("acc_score2 : ", acc_score1)


    2) SVM - SVC, SVR

In [11]:
from sklearn.svm import SVR
from sklearn.svm import SVC

print("----------SVR----------")

rd_df_x = df_need_2[[SOLAR]]
rd_df_y = df_need_2[["watt"]]

rd_df_x_train, rd_df_x_test, rd_df_y_train, rd_df_y_test = train_test_split(rd_df_x, rd_df_y, test_size=0.2, shuffle=True, random_state = 42)

model = SVR()
model.fit(rd_df_x_train, rd_df_y_train)
train_score1 = model.score(rd_df_x_train, rd_df_y_train)
test_score1 = model.score(rd_df_x_test, rd_df_y_test)
pred1 = model.predict(rd_df_x_test)
# acc_score1 = accuracy_score(rd_df_y_test, pred1)

print("훈련정확도 : ", train_score1)
print("예측정확도 : ", test_score1)
# print("acc_score : ", acc_score1)


rd_df_x = df_need_2[[SOLAR, "최저기온", "최고기온", "일강수량", "평균풍속", "평균상대습도", "가조시간", "평균전운량"]]
rd_df_y = df_need_2[["watt"]]

rd_df_x_train, rd_df_x_test, rd_df_y_train, rd_df_y_test = train_test_split(rd_df_x, rd_df_y, test_size=0.2, shuffle=True, random_state = 42)

model = SVR()
model.fit(rd_df_x_train, rd_df_y_train)
train_score2 = model.score(rd_df_x_train, rd_df_y_train)
test_score2 = model.score(rd_df_x_test, rd_df_y_test)
pred2 = model.predict(rd_df_x_test)
# acc_score2 = accuracy_score(rd_df_y_test, pred2)

print("훈련정확도2 : ", train_score2)
print("예측정확도2 : ", test_score2)
# print("acc_score2 : ", acc_score1)




----------SVR----------
훈련정확도 :  0.44033092403581464
예측정확도 :  0.43397978129214965
훈련정확도2 :  0.43686707462862073
예측정확도2 :  0.4314148312144176


    3) GradientBoosting

In [12]:

# rd_df_x = df_need_2[[SOLAR, "최저기온", "최고기온", "일강수량", "평균풍속", "평균상대습도", "가조시간", "평균전운량"]]
# rd_df_y = df_need_2[["watt"]]

# rd_df_x_train, rd_df_x_test, rd_df_y_train, rd_df_y_test = train_test_split(rd_df_x, rd_df_y, test_size=0.2, shuffle=True, random_state = 42)

# model = GradientBoostingRegressor()
# model.fit(rd_df_x_train, rd_df_y_train)
# train_score2 = model.score(rd_df_x_train, rd_df_y_train)
# test_score2 = model.score(rd_df_x_test, rd_df_y_test)
# pred2 = model.predict(rd_df_x_test)
# # acc_score2 = accuracy_score(rd_df_y_test, pred2)

# print("훈련 R-squared1: ", train_score2)
# print("예측 R-squared1 : ", test_score2)

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier

print("----------GradientBoosting----------")

----------GradientBoosting----------


In [13]:
rd_df_x = df_need_2[["match_solar","평균풍속", "평균상대습도", "가조시간"]]
rd_df_y = df_need_2[["watt"]]

rd_df_x_train, rd_df_x_test, rd_df_y_train, rd_df_y_test = train_test_split(rd_df_x, 
                                                                            rd_df_y, 
                                                                            test_size=0.2, 
                                                                            shuffle=True,
                                                                            random_state = 42)

model = GradientBoostingRegressor()
model.fit(rd_df_x_train, rd_df_y_train)
train_score1 = model.score(rd_df_x_train, rd_df_y_train)
test_score1 = model.score(rd_df_x_test, rd_df_y_test)
pred1 = model.predict(rd_df_x_test)

print("훈련 R-squared1 : ", train_score1)
print("예측 R-squared1 : ", test_score1)

훈련 R-squared1 :  0.879130532048404
예측 R-squared1 :  0.8336092148885401


In [18]:
y_true_list=[]
y_pred_list=[]
for i, j in zip(rd_df_y_test['watt'], range(len(pred1))):
    if (i !=0) & (pred1[j]>-3):
        y_true_list.append(i)
        y_pred_list.append(pred1[j])
        
y_true_list, y_pred_list = np.array(y_true_list), np.array(y_pred_list)

np.mean(np.abs((y_true_list - y_pred_list) / y_true_list)) *100

23.77415618549664

In [19]:
len(y_pred_list)

690

In [20]:
s1 = pd.Series(y_true_list)
s2 = pd.Series(y_pred_list)

dataaa = {"true" : s1, "pred" : s2}

df12 = pd.DataFrame(dataaa)
df12["diff2"] = (df12["true"] - df12["pred"]) / df12["true"]

# plt.plot(df12["true"])
# plt.plot(df12["pred"])
df12.describe()

,true,pred,diff2
count,690.000000,690.000000,690.000000
mean,2613.257454,2579.624597,-0.071331
std,1640.461358,1547.030102,0.523382
min,45.077460,177.569826,-7.187008
25%,1223.657820,1257.936323,-0.162696
50%,2351.988648,2330.196687,-0.001181
75%,3686.006069,3642.023378,0.132099
max,8808.159104,8006.368307,0.720616


In [22]:
lists= []
for i in df12["diff2"]:
    if i > -3:
        lists.append(i)
np.mean(np.abs(lists)) *100, len(lists)

(21.030962753321056, 686)

In [27]:
rd_df_x = df_need_2[['env_slopesolar', "최저기온", "최고기온", "일강수량", "평균풍속", "평균상대습도", "가조시간", "평균전운량"]]
rd_df_y = df_need_2[["watt"]]

rd_df_x_train, rd_df_x_test, rd_df_y_train, rd_df_y_test = train_test_split(rd_df_x, rd_df_y, test_size=0.2, shuffle=True, random_state = 42)

model = GradientBoostingRegressor()
model.fit(rd_df_x_train, rd_df_y_train)
train_score2 = model.score(rd_df_x_train, rd_df_y_train)
test_score2 = model.score(rd_df_x_test, rd_df_y_test)
pred2 = model.predict(rd_df_x_test)
# acc_score2 = accuracy_score(rd_df_y_test, pred2)

print("훈련 R-squared1: ", train_score2)
print("예측 R-squared1 : ", test_score2)

훈련 R-squared1:  0.9098835246535839
예측 R-squared1 :  0.8737779498282361


In [28]:
y_true_list=[]
y_pred_list=[]
for i, j in zip(rd_df_y_test['watt'], range(len(pred2))):
    if i !=0:
        y_true_list.append(i)
        y_pred_list.append(pred1[j])
        
y_true_list, y_pred_list = np.array(y_true_list), np.array(y_pred_list)

# np.mean(np.abs((y_true_list - y_pred_list) / y_true_list)) *100

In [31]:
s1 = pd.Series(y_true_list)
s2 = pd.Series(y_pred_list)

dataaa = {"true" : s1, "pred" : s2}

df12 = pd.DataFrame(dataaa)
df12["diff2"] = (df12["true"] - df12["pred"]) / df12["true"]

# plt.plot(df12["true"])
# plt.plot(df12["pred"])
df12.describe()

,true,pred,diff2
count,690.000000,690.000000,690.000000
mean,2613.257454,2579.624597,-0.071331
std,1640.461358,1547.030102,0.523382
min,45.077460,177.569826,-7.187008
25%,1223.657820,1257.936323,-0.162696
50%,2351.988648,2330.196687,-0.001181
75%,3686.006069,3642.023378,0.132099
max,8808.159104,8006.368307,0.720616


In [32]:
lists= []
for i in df12["diff2"]:
    if i > -3:
        lists.append(i)
np.mean(np.abs(lists)) *100, len(lists)

(21.030962753321056, 686)

    4) Decisiontree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

print("----------DecisionTree----------")

rd_df_x = df_need_2[[SOLAR]]
rd_df_y = df_need_2[["watt"]]

rd_df_x_train, rd_df_x_test, rd_df_y_train, rd_df_y_test = train_test_split(rd_df_x, rd_df_y, test_size=0.2, shuffle=True, random_state = 42)

model = DecisionTreeRegressor()
model.fit(rd_df_x_train, rd_df_y_train)
train_score1 = model.score(rd_df_x_train, rd_df_y_train)
test_score1 = model.score(rd_df_x_test, rd_df_y_test)
pred1 = model.predict(rd_df_x_test)
# acc_score1 = accuracy_score(rd_df_y_test, pred1)

print("훈련정확도 : ", train_score1)
print("예측정확도 : ", test_score1)
# print("acc_score : ", acc_score1)


rd_df_x = df_need_2[[SOLAR, "최저기온", "최고기온", "일강수량", "평균풍속", "평균상대습도", "가조시간", "평균전운량"]]
rd_df_y = df_need_2[["watt"]]

rd_df_x_train, rd_df_x_test, rd_df_y_train, rd_df_y_test = train_test_split(rd_df_x, rd_df_y, 
                                                                            test_size=0.2, 
                                                                            shuffle=True, 
                                                                            random_state = 42)

model = DecisionTreeRegressor()
model.fit(rd_df_x_train, rd_df_y_train)
train_score2 = model.score(rd_df_x_train, rd_df_y_train)
test_score2 = model.score(rd_df_x_test, rd_df_y_test)
pred2 = model.predict(rd_df_x_test)
# acc_score2 = accuracy_score(rd_df_y_test, pred2)

print("훈련정확도2 : ", train_score2)
print("예측정확도2 : ", test_score2)
# print("acc_score2 : ", acc_score1)




    5) Randomforest

In [34]:


# rd_df_x = df_need_2[[SOLAR, "최저기온", "최고기온", "일강수량", "평균풍속", "평균상대습도", "가조시간", "평균전운량"]]
# rd_df_y = df_need_2[["watt"]]

# rd_df_x_train, rd_df_x_test, rd_df_y_train, rd_df_y_test = train_test_split(rd_df_x, rd_df_y, test_size=0.2, shuffle=True, random_state = 42)

# model = RandomForestRegressor()
# model.fit(rd_df_x_train, rd_df_y_train)
# train_score2 = model.score(rd_df_x_train, rd_df_y_train)
# test_score2 = model.score(rd_df_x_test, rd_df_y_test)
# pred2 = model.predict(rd_df_x_test)
# # acc_score2 = accuracy_score(rd_df_y_test, pred2)

# print("훈련 R-squared2 : ", train_score2)
# print("예측 R-squared2 : ", test_score2)


from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

print("----------RandomForest----------")

----------RandomForest----------


In [35]:
rd_df_x = df_need_2[["reset_match","평균풍속", "평균상대습도", "가조시간"]]
rd_df_y = df_need_2[["watt"]]

rd_df_x_train, rd_df_x_test, rd_df_y_train, rd_df_y_test = train_test_split(rd_df_x, 
                                                                            rd_df_y, 
                                                                            test_size=0.2, 
                                                                            shuffle=True, 
                                                                            random_state = 17)

model = RandomForestRegressor()
model.fit(rd_df_x_train, rd_df_y_train)
train_score1 = model.score(rd_df_x_train, rd_df_y_train)
test_score1 = model.score(rd_df_x_test, rd_df_y_test)
pred1 = model.predict(rd_df_x_test)

print("훈련 R-squared1 : ", train_score1)
print("예측 R-squared1 : ", test_score1)

훈련 R-squared1 :  0.9785522361845286
예측 R-squared1 :  0.8435036980578434


In [36]:
y_true_list=[]
y_pred_list=[]
for i, j in zip(rd_df_y_test['watt'], range(len(pred1))):
    if i !=0:
        y_true_list.append(i)
        y_pred_list.append(pred1[j])
        
y_true_list, y_pred_list = np.array(y_true_list), np.array(y_pred_list)

np.mean(np.abs((y_true_list - y_pred_list) / y_true_list)) * 100

27.98209890609009

In [37]:
s1 = pd.Series(y_true_list)
s2 = pd.Series(y_pred_list)

dataaa = {"true" : s1, "pred" : s2}

df12 = pd.DataFrame(dataaa)
df12["diff2"] = (df12["true"] - df12["pred"]) / df12["true"]

# plt.plot(df12["true"])
# plt.plot(df12["pred"])
df12.describe()

lists= []
for i in df12["diff2"]:
    if i > -3:
        lists.append(i)
np.mean(np.abs(lists)) *100, len(lists)

(19.97835625597008, 685)

In [38]:
rd_df_x = df_need_2[['reset_match', "최저기온", "최고기온", "일강수량", "평균풍속", "평균상대습도", "가조시간", "평균전운량"]]
rd_df_y = df_need_2[["watt"]]

rd_df_x_train, rd_df_x_test, rd_df_y_train, rd_df_y_test = train_test_split(rd_df_x, rd_df_y, test_size=0.2, shuffle=True, random_state = 42)

model = RandomForestRegressor()
model.fit(rd_df_x_train, rd_df_y_train)
train_score2 = model.score(rd_df_x_train, rd_df_y_train)
test_score2 = model.score(rd_df_x_test, rd_df_y_test)
pred2 = model.predict(rd_df_x_test)
# acc_score2 = accuracy_score(rd_df_y_test, pred2)

print("훈련 R-squared2 : ", train_score2)
print("예측 R-squared2 : ", test_score2)


훈련 R-squared2 :  0.9796620537257457
예측 R-squared2 :  0.8434528680352378


In [39]:
y_true_list=[]
y_pred_list=[]
for i, j in zip(rd_df_y_test['watt'], range(len(pred2))):
    if i !=0:
        y_true_list.append(i)
        y_pred_list.append(pred1[j])
        
y_true_list, y_pred_list = np.array(y_true_list), np.array(y_pred_list)

np.mean(np.abs((y_true_list - y_pred_list) / y_true_list)) * 100

146.0457614691518

In [40]:
s1 = pd.Series(y_true_list)
s2 = pd.Series(y_pred_list)

dataaa = {"true" : s1, "pred" : s2}

df12 = pd.DataFrame(dataaa)
df12["diff2"] = (df12["true"] - df12["pred"]) / df12["true"]

# plt.plot(df12["true"])
# plt.plot(df12["pred"])
df12.describe()

lists= []
for i in df12["diff2"]:
    if i > -3:
        lists.append(i)
np.mean(np.abs(lists)) *100, len(lists)

(64.3175919698319, 628)